In [43]:
import time
import jwt
import requests

# 实际KEY，过期时间
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)

    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )


In [44]:
# 打开并读取文本文件
filename = 'article.txt'
content=[]
# 使用with语句确保文件在使用完毕后会被自动关闭
with open(filename, 'r', encoding='utf-8') as file:
    # 逐行读取文件内容
    for line in file:
        line=line.strip()
        if line and line!="图片" and line!="关注该公众号" and "\u200b" not in line:
            content.append(line)

print(content[10])

其实身边那么多优秀的例子，取得了很好的平衡的人。我也不是贱或者懒不去学他们，这事到底咋说理呢？哈哈，想到这我都笑了，反正没成为他们那样，错的肯定不是别人，指定是我😂嗨，整得还挺无解的，倒让我又释怀挺多。不错，慢慢看吧，走一步是一步；快快看吧，跑一步是一步。如果自己承受能力还不错，就尽量别人身边人吃亏吧，是吧，想不清楚要什么但还算善良，那努努力拼一拼，先别膈应别人，自己以后再看，一边做再一边思考吧🤔。


In [45]:
articles=[]
idx=0
articles.append({
    "title":"",
    "meta":"",
    "lines":[]
})

for line in content:
    if line =="微信扫一扫":
        idx+=1
        articles.append({
            "title":"",
            "meta":"",
            "lines":[]
        })
        continue
    else:
        if articles[idx]["title"] =="":
            articles[idx]["title"]=line
        elif articles[idx]["meta"] =="":
            articles[idx]["meta"]=line
        else:
            articles[idx]["lines"].append(line)

            
    

In [46]:
for article in articles:
    article["content"]=''.join(article["lines"])

In [47]:
articles[100]

{'title': '亚裔不受欢迎',
 'meta': '原创 兽妖 半业余爱好者 2021年05月05日 17:14',
 'lines': ['近些日子，自己的生活太美满，没有什么忧愁迷茫要大家分担，所以聊一聊外界事物。',
  'Slap an Asian Chalenge是一两个月之前的事情了，亚裔不好受，我想到了一些东西，就我个人两次在国外的短暂经历而言，总体感觉没有什么特别，主要还是一个新鲜感，其次还是有些生疏与不自在。',
  '高中时去过一次新加坡，在一个商场里买东西排队，被一个老太太说中文批评：“你看不见上面那个标志吗？这是老年人专用排队通道”，我一看，还真有这么一个标志，国内还真没遇到过。我不好意思的让开，那老太太还怪里怪气加了一句：“从中国来的吧 ”',
  "大二去了一次澳大利亚墨尔本，和同学在麦当劳里吃东西讨论事情。一个女同学吃完了，趴在桌上休息。服务员马上就走了过来，对她说'It's not allowed to sleep here'，还说了些什么，总体意思是不允许流浪汉在这过夜。可我看我那同学穿的也挺得体的。",
  '要我去评价这两个事件，也没有什么好说的，光是回想，都让人感觉有些拘促难言。也许也是我英语水平不够好，没能完全适应国外的生活。',
  '不得不说，在国外的经历，以及了解到的新闻，让我感觉西方世界的人和我们还是有很多不一样的。',
  '最有感觉的，就从 游行 这回事开始说起吧，再延伸到其他方面。',
  '国外似乎总是游行，为了本国的人，本国的事务，本国的动物，外国的人，外国的事务，外国的动物。情况便是一群人，充满热情地聚集在了一起，在大街上，拉着横幅，举着牌子，走那么一段距离。',
  '仔细想想，他们也知道，游行那一段距离，他们并没有改变什么，他们追求的是发声，为他们关心的某一件事情发声。可能是为了宣传，可能是为了对抗。且不说他们的关心到底是什么程度的关心，他们这种聚集的行为就让我感觉很奇妙，不说效果，至少有很多人是愿意为一些事情走上街头，报之以关注的。',
  '很难想象，也是这样的地方，也充满了恶言恶语。我已经不知道该如何看代他们对亚裔的偏见了，这又是一回事了。',
  '我该如何看代外国人对亚裔的偏见歧视呢？这是个问题，是问我的问题，问一个不在国外的中国人的问题。我能否为他国的亚裔走上中国街头，为他们造势发

In [48]:
import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

In [49]:
questions=[
    {
        "question":"为什么感到焦虑和迷茫",
        "answer": "",
        "reference": ""
    },
        {
        "question":"压力来源于哪里",
        "answer": "",
        "reference": ""
    },
       {
        "question":"客户有什么求而不得的东西",
        "answer": "",
        "reference": ""
    }
]

In [50]:
def ask_glm(content):
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
      'Content-Type': 'application/json',
      'Authorization': generate_token("your_token", 1000)
    }

    data = {
        "model": "glm-3-turbo",
        "messages": [{"role": "user", "content": content}]
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [51]:
questions

[{'question': '为什么感到焦虑和迷茫', 'answer': '', 'reference': ''},
 {'question': '压力来源于哪里', 'answer': '', 'reference': ''},
 {'question': '客户有什么求而不得的东西', 'answer': '', 'reference': ''}]

In [52]:
del articles[101]

In [53]:
articles[100]

{'title': '亚裔不受欢迎',
 'meta': '原创 兽妖 半业余爱好者 2021年05月05日 17:14',
 'lines': ['近些日子，自己的生活太美满，没有什么忧愁迷茫要大家分担，所以聊一聊外界事物。',
  'Slap an Asian Chalenge是一两个月之前的事情了，亚裔不好受，我想到了一些东西，就我个人两次在国外的短暂经历而言，总体感觉没有什么特别，主要还是一个新鲜感，其次还是有些生疏与不自在。',
  '高中时去过一次新加坡，在一个商场里买东西排队，被一个老太太说中文批评：“你看不见上面那个标志吗？这是老年人专用排队通道”，我一看，还真有这么一个标志，国内还真没遇到过。我不好意思的让开，那老太太还怪里怪气加了一句：“从中国来的吧 ”',
  "大二去了一次澳大利亚墨尔本，和同学在麦当劳里吃东西讨论事情。一个女同学吃完了，趴在桌上休息。服务员马上就走了过来，对她说'It's not allowed to sleep here'，还说了些什么，总体意思是不允许流浪汉在这过夜。可我看我那同学穿的也挺得体的。",
  '要我去评价这两个事件，也没有什么好说的，光是回想，都让人感觉有些拘促难言。也许也是我英语水平不够好，没能完全适应国外的生活。',
  '不得不说，在国外的经历，以及了解到的新闻，让我感觉西方世界的人和我们还是有很多不一样的。',
  '最有感觉的，就从 游行 这回事开始说起吧，再延伸到其他方面。',
  '国外似乎总是游行，为了本国的人，本国的事务，本国的动物，外国的人，外国的事务，外国的动物。情况便是一群人，充满热情地聚集在了一起，在大街上，拉着横幅，举着牌子，走那么一段距离。',
  '仔细想想，他们也知道，游行那一段距离，他们并没有改变什么，他们追求的是发声，为他们关心的某一件事情发声。可能是为了宣传，可能是为了对抗。且不说他们的关心到底是什么程度的关心，他们这种聚集的行为就让我感觉很奇妙，不说效果，至少有很多人是愿意为一些事情走上街头，报之以关注的。',
  '很难想象，也是这样的地方，也充满了恶言恶语。我已经不知道该如何看代他们对亚裔的偏见了，这又是一回事了。',
  '我该如何看代外国人对亚裔的偏见歧视呢？这是个问题，是问我的问题，问一个不在国外的中国人的问题。我能否为他国的亚裔走上中国街头，为他们造势发

In [54]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("./m3e-base")

In [55]:
question_sentences = [x['question'] for x in questions]
pdf_content_sentences = [x['content'] for x in articles]

question_embeddings = model.encode(question_sentences, normalize_embeddings=True)
pdf_embeddings = model.encode(pdf_content_sentences, normalize_embeddings=True)




In [56]:
for query_idx, feat in enumerate(question_embeddings):
    score = feat @ pdf_embeddings.T
    max3_score_page_idx = score.argsort()[-3:][::-1]
    
    prompt = '''你是一个心理专家助手，擅长通过客户的日记文本来分析客户的心理状态并给出相关建议。帮我结合给定的日记内容，回答一个问题。如果问题无法从资料中获得，请输出结合给定的资料，无法回答问题。
日记1标题：{0}
日记1内容：{1}

日记2标题：{2}
日记2内容：{3}

日记3标题：{4}
日记3内容：{5}

问题：{6}
    '''.format(
        articles[max3_score_page_idx[0]]['title'],
        articles[max3_score_page_idx[0]]['content'],
        articles[max3_score_page_idx[1]]['title'],
        articles[max3_score_page_idx[1]]['content'],
        articles[max3_score_page_idx[2]]['title'],
        articles[max3_score_page_idx[2]]['content'],
        questions[query_idx]["question"]
    )
    questions[query_idx]['max3_score_page_idx']=max3_score_page_idx;
    questions[query_idx]['prompt']=prompt
print('ok')

ok


In [57]:
for question in questions:
    answer = ask_glm(question['prompt'])['choices'][0]['message']['content']
    question['answer']=answer
print('ok')
    

ok


In [58]:
for question in questions:
    print("提问:",question['question'])
    print("参考日记：",'#'.join([("("+str(i)+")"+articles[i]['title']) for i in question['max3_score_page_idx']]))
    print("回答",question['answer'],'\n\n\n')

提问: 为什么感到焦虑和迷茫
参考日记： (3)很无语#(6)24岁我在害怕什么？#(5)如饥似渴的焦虑
回答 从您提供的日记内容中，可以看出您感到焦虑和迷茫的原因主要有以下几点：

1. **社会和个人的期待**：您提到身边的人都已经思路清晰有条不紊地前进，而自己还在害怕和迷茫中挣扎，这种比较产生了无语和无助的感觉。

2. **自我价值的确认**：尽管您有很多外在的成就，如高薪、好的学历、健康等，但您感觉这些成就并没有给您带来预期的满足感。您认为这些成就本应该是与生俱来的，而不是通过努力获得的，这种认知导致您感到焦虑。

3. **未来的不确定性**：对于即将到来的出国留学生活，您既有憧憬也有担忧。您害怕在美国听不懂课、学不到技术知识、找不到实习和工作，担心自己会回到国内后的一切从头再来。

4. **自我认同的探索**：您不断问自己“我到底要成为一个什么样的人？”感到迷茫和焦虑。您希望找到一种内心的肯定和自信，而不是仅仅通过外在成就来衡量自我价值。

5. **社会的比较与压力**：您感觉到身边的人都十分自信并且似乎不需要经历困惑就能找到答案，这给您带来了压力和不安。同时，您认为社会对自信的人有偏见，这使得您在表达自我价值和尝试新事物时感到犹豫。

建议您可以通过以下方式来缓解焦虑和迷茫：

- **自我接纳**：接受自己的不完美和目前的状态，认识到每个人的人生节奏和经历都是不同的。

- **心理咨询**：如果焦虑和迷茫感持续影响到您的生活质量，可以考虑寻求专业的心理咨询帮助。

- **明确目标**：尝试明确自己的长期和短期目标，并制定实现这些目标的计划和步骤。

- **积极生活态度**：培养积极的生活态度，学会从挑战中寻找成长的机会。

- **社交支持**：与家人和朋友交流您的感受，他们的支持和理解可以帮助您减轻负担。

- **专注当下**：通过冥想、瑜伽或其他放松技巧来帮助自己专注当下，减少对未来的过度担忧。

请记住，焦虑和迷茫是成长过程中正常的心理现象，通过积极的方法可以逐渐克服。 



提问: 压力来源于哪里
参考日记： (18)抗压，不怎么抗压#(27)谈论一下工作与生活的平衡#(28)谈论一下工作与生活的平衡
回答 根据您提供的日记内容，可以分析出压力的来源主要集中在以下几个方面：

1. **工作环境的压力**：日记中反复提到工作中

In [105]:
emotions=list(map(lambda e:{'emotion':e},[
    '焦虑','迷茫','压力','急躁','开心','期待','悲伤','愤怒','恐惧','烦躁','无聊','疲惫','孤独','失落','绝望','悲观'
]))

In [60]:
emotions

[{'emotion': '焦虑'},
 {'emotion': '迷茫'},
 {'emotion': '压力'},
 {'emotion': '急躁'},
 {'emotion': '开心'},
 {'emotion': '期待'},
 {'emotion': '悲伤'},
 {'emotion': '愤怒'},
 {'emotion': '恐惧'},
 {'emotion': '烦躁'},
 {'emotion': '无聊'},
 {'emotion': '疲惫'},
 {'emotion': '孤独'},
 {'emotion': '失落'},
 {'emotion': '绝望'},
 {'emotion': '悲观'}]

In [61]:
emitions_embenings = model.encode([e['emotion'] for e in emotions], normalize_embeddings=True)

In [106]:
for query_idx, feat in enumerate(emitions_embenings):
    score = feat @ pdf_embeddings.T
    relevance_ranks = score.argsort()[::-1]
    # print(relevance_ranks)
    emotions[query_idx]['relevance_ranks']=relevance_ranks
    emotions[query_idx]['score_ranks']=sorted(score,reverse=True)
print('ok')

ok


In [107]:
emotions

[{'emotion': '焦虑',
  'relevance_ranks': array([  5,   6,  18,   3,   4,  75,  73,  43,  74,  23,  59,  36,  65,
           0,  28,  27,  87,  42,  64,  63,  47,  26,  66,  52,  49,  25,
          56,  16,   9,  17,  38,  30,  12,  20, 100,  40,   7,  57,  76,
          53,  70,  71,  62,  19,  31,  48,  60,  81,  55,  69,  92,  32,
          89,  21,  11,  82,  22,  80,  37,  45,  72,  44,  91,  34,  29,
          88,  67,  35,  98,  10,  86,  78,  77,  13,  15,  24,  33,  46,
          61,  14,  54,  97,  93,   1,  99,  58,  95,  79,   8,  68,  41,
          94,  96,  83,  50,  51,   2,  90,  85,  39,  84]),
  'score_ranks': [0.7992048,
   0.79801786,
   0.78861547,
   0.7848884,
   0.76528025,
   0.76034206,
   0.7590598,
   0.75322974,
   0.7497765,
   0.7495538,
   0.747592,
   0.74743116,
   0.74540776,
   0.74281496,
   0.7414573,
   0.7414573,
   0.74056494,
   0.7392965,
   0.7391403,
   0.7380763,
   0.7357977,
   0.7336286,
   0.73313093,
   0.73037875,
   0.7299318,
   0.729

In [80]:
articles[19]

{'title': '无聊的',
 'meta': '兽妖 半业余爱好者 2023年11月11日 00:20 浙江',
 'lines': ['以前可能觉得自己挺有趣的，现在觉得自己真的挺无聊的，别人看起来无聊，自己也觉得无聊'],
 'content': '以前可能觉得自己挺有趣的，现在觉得自己真的挺无聊的，别人看起来无聊，自己也觉得无聊'}